## Load Packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import bayesflow as bf
import os
from numba import njit
from sklearn.preprocessing import StandardScaler
import numba as nb
import copy
import random
from tensorflow.keras import backend as K
import gc
from pathlib import Path
from tensorflow.keras.utils import to_categorical
from tensorflow_probability import distributions as tfd
import tensorflow as tf
import ipynbname
RNG = np.random.default_rng(2023)
from diffusion_functions import diffusion_trial, diffusion_prior, generate_condition_matrix, diffusion_configurator
# Suppress scientific notation for floats
np.set_printoptions(suppress=True)



C:\Users\u0145642\AppData\Roaming\Python\Python311\site-packages\bayesflow\trainers.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Get file path

In [2]:
notebook_path = Path(ipynbname.path()).resolve().parent
root_dir = notebook_path.parent
#print(root_dir)

## Checkpoint

In [5]:
checkpoint_path =  root_dir/ "networks/standard"
checkpoint_path_t_df1 = root_dir/ "networks/t_df1"
checkpoint_path_t_df3 = root_dir/ "networks/t_df3"
checkpoint_path_t_df5 = root_dir/ "networks/t_df5"
checkpoint_path_uniform = root_dir/ "networks/uniform"

## Set up prior and condition matrix

In [6]:
PARAM_NAMES = [
    "Drift rate 1",
    "Drift rate 2",
    "Boundary separation",
    "Response bias",
    "Non-decision time"
]

prior = bf.simulation.Prior(prior_fun=diffusion_prior, param_names=PARAM_NAMES)

MIN_OBS = 100
MAX_OBS = 1000
NUM_CONDITIONS=2


def random_num_obs(min_obs=MIN_OBS, max_obs=MAX_OBS):
   """Draws a random number of observations for all simulations in a batch."""

   return RNG.integers(low=min_obs, high=max_obs + 1)


context_gen = bf.simulation.ContextGenerator(
    non_batchable_context_fun=random_num_obs,
    batchable_context_fun=generate_condition_matrix,
    use_non_batchable_for_batchable=True
)

## Set up and train the DDM estimators

### Standard estimator

In [13]:
@nb.jit(nopython=True)
def diffusion_experiment(theta, design_matrix, num_obs, rng=None, *args):
    
    out = np.zeros((num_obs, 2))
    for n in range(num_obs):
        index = design_matrix[n]
        rt,resp = diffusion_trial(theta[index], theta[-3], theta[-2], theta[-1])
        out[n, :] = np.array([rt,resp])
        
    out[:,0] = np.log(out[:,0])
    
    return out

simulator = bf.simulation.Simulator(simulator_fun=diffusion_experiment, context_generator=context_gen)

model = bf.simulation.GenerativeModel(prior=prior, simulator=simulator)

summary_net = bf.networks.SetTransformer(input_dim=4, summary_dim=12)

inference_net = bf.networks.InvertibleNetwork(
    num_params=len(prior.param_names),
    coupling_design="interleaved",
    name="ddm_inference",
)

amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net)

trainer = bf.trainers.Trainer(
    generative_model=model, amortizer=amortizer, configurator=diffusion_configurator
    ,checkpoint_path = checkpoint_path
)
amortizer.summary()

#history = trainer.train_online(epochs=100, iterations_per_epoch=1000, batch_size=32)

INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 5)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 592, 2)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:Shape of simulation non-batchable context: ()
INFO:root:Could not determine shape of simulation batchable context. Type appears to be non-array: <class 'list'>,                                    so make sure your input configurator takes cares of that!
INFO:root:Loaded loss history from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\standard\history_100.pkl.
INFO:root:Networks loaded from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\standard\ckpt-100
INFO:root:Performing a consistency check with provided components...
INFO:root:Don

Model: "amortized_posterior"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ddm_inference (InvertibleN  multiple                  434037    
 etwork)                                                         
                                                                 
 set_transformer (SetTransf  multiple                  46544     
 ormer)                                                          
                                                                 
Total params: 480581 (1.83 MB)
Trainable params: 480521 (1.83 MB)
Non-trainable params: 60 (240.00 Byte)
_________________________________________________________________


### Robust estimator (t, df=1)

In [14]:
@nb.jit(nopython=True)

def diffusion_experiment_contaminated_t_df1(theta, design_matrix, num_obs, rng=None, *args):
    out = np.zeros((num_obs, 2))
    for n in range(num_obs):
        index = design_matrix[n]
        rt,resp = diffusion_trial(theta[index], theta[-3], theta[-2], theta[-1])
        out[n, :] = np.array([rt,resp])
    
    #log transformation
    out[:,0] = np.log(out[:,0])
    
    CC=np.random.standard_t(df=1,size=num_obs)
    X=np.random.binomial(n=1,p=.1,size=num_obs)
    out[:,0] = (1-X)*out[:,0] + (X)*np.log(np.abs(CC))
    out[:,1] = (1-X)*out[:,1] + (X)*np.random.binomial(n=1,p=0.5,size=num_obs)
    
    return out

simulator_t_df1 = bf.simulation.Simulator(simulator_fun=diffusion_experiment_contaminated_t_df1, context_generator=context_gen)

model_t_df1 = bf.simulation.GenerativeModel(prior=prior, simulator=simulator_t_df1)

#SetTransformer() is a permutation invariant network
#input_dim = how many dimensions the configured data would have
#here we have a RT, dummy coded variable for the two choices (one or two?), the context dummy
summary_net_t_df1 = bf.networks.SetTransformer(input_dim=4, summary_dim=12)

#we are turning off the kernel and dropout regularization for the networks 
#since we don’t need these for online training
inference_net_t_df1 = bf.networks.InvertibleNetwork(
    num_params=len(prior.param_names),
    coupling_design="interleaved",
    name="ddm_inference",
)

amortizer_t_df1 = bf.amortizers.AmortizedPosterior(inference_net_t_df1, summary_net_t_df1)


trainer_t_df1 = bf.trainers.Trainer(
    generative_model=model_t_df1, amortizer=amortizer_t_df1, configurator=diffusion_configurator,
    checkpoint_path = checkpoint_path_t_df1
)
amortizer_t_df1.summary()

#history = trainer_c_t_df1.train_online(epochs=100, iterations_per_epoch=1000, batch_size=32)

INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 5)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 786, 2)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:Shape of simulation non-batchable context: ()
INFO:root:Could not determine shape of simulation batchable context. Type appears to be non-array: <class 'list'>,                                    so make sure your input configurator takes cares of that!
INFO:root:Loaded loss history from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\t_df1\history_100.pkl.
INFO:root:Networks loaded from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\t_df1\ckpt-100
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


Model: "amortized_posterior_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ddm_inference (InvertibleN  multiple                  434037    
 etwork)                                                         
                                                                 
 set_transformer_1 (SetTran  multiple                  46544     
 sformer)                                                        
                                                                 
Total params: 480581 (1.83 MB)
Trainable params: 480521 (1.83 MB)
Non-trainable params: 60 (240.00 Byte)
_________________________________________________________________


### Robust estimator (t, df=3)

In [15]:
@nb.jit(nopython=True)

def diffusion_experiment_contaminated_t_df3(theta, design_matrix, num_obs, rng=None, *args):
    out = np.zeros((num_obs, 2))
    for n in range(num_obs):
        index = design_matrix[n]
        rt,resp = diffusion_trial(theta[index], theta[-3], theta[-2], theta[-1])
        out[n, :] = np.array([rt,resp])
    
    #log transformation
    out[:,0] = np.log(out[:,0])
    
    CC=np.random.standard_t(df=3,size=num_obs)
    X=np.random.binomial(n=1,p=.1,size=num_obs)
    out[:,0] = (1-X)*out[:,0] + (X)*np.log(np.abs(CC))
    out[:,1] = (1-X)*out[:,1] + (X)*np.random.binomial(n=1,p=0.5,size=num_obs)
    
    return out

simulator_t_df3 = bf.simulation.Simulator(simulator_fun=diffusion_experiment_contaminated_t_df3, context_generator=context_gen)

model_t_df3 = bf.simulation.GenerativeModel(prior=prior, simulator=simulator_t_df3)

#SetTransformer() is a permutation invariant network
#input_dim = how many dimensions the configured data would have
#here we have a RT, dummy coded variable for the two choices (one or two?), the context dummy
summary_net_t_df3 = bf.networks.SetTransformer(input_dim=4, summary_dim=12)

#we are turning off the kernel and dropout regularization for the networks 
#since we don’t need these for online training
inference_net_t_df3 = bf.networks.InvertibleNetwork(
    num_params=len(prior.param_names),
    coupling_design="interleaved",
    name="ddm_inference",
)

amortizer_t_df3 = bf.amortizers.AmortizedPosterior(inference_net_t_df3, summary_net_t_df3)


trainer_t_df3 = bf.trainers.Trainer(
    generative_model=model_t_df3, amortizer=amortizer_t_df3, configurator=diffusion_configurator,
    checkpoint_path = checkpoint_path_t_df3
)
amortizer_t_df3.summary()

#history = trainer_c_t_df3.train_online(epochs=100, iterations_per_epoch=1000, batch_size=32)

INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 5)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 626, 2)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:Shape of simulation non-batchable context: ()
INFO:root:Could not determine shape of simulation batchable context. Type appears to be non-array: <class 'list'>,                                    so make sure your input configurator takes cares of that!
INFO:root:Loaded loss history from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\t_df3\history_100.pkl.
INFO:root:Networks loaded from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\t_df3\ckpt-100
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


Model: "amortized_posterior_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ddm_inference (InvertibleN  multiple                  434037    
 etwork)                                                         
                                                                 
 set_transformer_2 (SetTran  multiple                  46544     
 sformer)                                                        
                                                                 
Total params: 480581 (1.83 MB)
Trainable params: 480521 (1.83 MB)
Non-trainable params: 60 (240.00 Byte)
_________________________________________________________________


### Robust estimator (t, df=5)

In [16]:
@nb.jit(nopython=True)

def diffusion_experiment_contaminated_t_df5(theta, design_matrix, num_obs, rng=None, *args):
    out = np.zeros((num_obs, 2))
    for n in range(num_obs):
        index = design_matrix[n]
        rt,resp = diffusion_trial(theta[index], theta[-3], theta[-2], theta[-1])
        out[n, :] = np.array([rt,resp])
    
    #log transformation
    out[:,0] = np.log(out[:,0])
    
    CC=np.random.standard_t(df=5,size=num_obs)
    X=np.random.binomial(n=1,p=.1,size=num_obs)
    out[:,0] = (1-X)*out[:,0] + (X)*np.log(np.abs(CC))
    out[:,1] = (1-X)*out[:,1] + (X)*np.random.binomial(n=1,p=0.5,size=num_obs)
    
    return out

simulator_t_df5 = bf.simulation.Simulator(simulator_fun=diffusion_experiment_contaminated_t_df5, context_generator=context_gen)

model_t_df5 = bf.simulation.GenerativeModel(prior=prior, simulator=simulator_t_df5)

#SetTransformer() is a permutation invariant network
#input_dim = how many dimensions the configured data would have
#here we have a RT, dummy coded variable for the two choices (one or two?), the context dummy
summary_net_t_df5 = bf.networks.SetTransformer(input_dim=4, summary_dim=12)

#we are turning off the kernel and dropout regularization for the networks 
#since we don’t need these for online training
inference_net_t_df5 = bf.networks.InvertibleNetwork(
    num_params=len(prior.param_names),
    coupling_design="interleaved",
    name="ddm_inference",
)

amortizer_t_df5 = bf.amortizers.AmortizedPosterior(inference_net_t_df5, summary_net_t_df5)


trainer_t_df5 = bf.trainers.Trainer(
    generative_model=model_t_df5, amortizer=amortizer_t_df5, configurator=diffusion_configurator,
    checkpoint_path = checkpoint_path_t_df5
)
amortizer_t_df5.summary()

#history = trainer_c_t_df5.train_online(epochs=100, iterations_per_epoch=1000, batch_size=32)

INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 5)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 907, 2)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:Shape of simulation non-batchable context: ()
INFO:root:Could not determine shape of simulation batchable context. Type appears to be non-array: <class 'list'>,                                    so make sure your input configurator takes cares of that!
INFO:root:Loaded loss history from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\t_df5\history_100.pkl.
INFO:root:Networks loaded from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\t_df5\ckpt-100
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


Model: "amortized_posterior_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ddm_inference (InvertibleN  multiple                  434037    
 etwork)                                                         
                                                                 
 set_transformer_3 (SetTran  multiple                  46544     
 sformer)                                                        
                                                                 
Total params: 480581 (1.83 MB)
Trainable params: 480521 (1.83 MB)
Non-trainable params: 60 (240.00 Byte)
_________________________________________________________________


### Robust estimator (U(0,20))

In [18]:
@nb.jit(nopython=True,parallel=True)
def diffusion_experiment_contaminated_uniform(theta, design_matrix, num_obs, rng=None, *args):
    
    out = np.zeros((num_obs, 2))
    for n in range(num_obs):
        index = design_matrix[n]
        rt,resp = diffusion_trial(theta[index], theta[-3], theta[-2], theta[-1])
        out[n, :] = np.array([rt,resp])
    
    out[:,0] = np.log(out[:,0])
    
    CC=np.random.uniform(0,20,size=num_obs)
    X=np.random.binomial(n=1,p=.1,size=num_obs)
    out[:,0] = (1-X)*out[:,0] + (X)*np.log(CC)
    out[:,1] = (1-X)*out[:,1] + (X)*np.random.binomial(n=1,p=0.5,size=num_obs)
    
    return out

simulator_uniform = bf.simulation.Simulator(simulator_fun=diffusion_experiment_contaminated_uniform, context_generator=context_gen)

model_uniform = bf.simulation.GenerativeModel(prior=prior, simulator=simulator_uniform)

summary_net_uniform = bf.networks.SetTransformer(input_dim=4, summary_dim=12)

#we are turning off the kernel and dropout regularization for the networks 
#since we don’t need these for online training
inference_net_uniform = bf.networks.InvertibleNetwork(
    num_params=len(prior.param_names),
    coupling_design="interleaved",
    name="ddm_inference",
)

amortizer_uniform = bf.amortizers.AmortizedPosterior(inference_net_uniform, summary_net_uniform)


trainer_uniform = bf.trainers.Trainer(
    generative_model=model_uniform, 
    amortizer=amortizer_uniform, 
    configurator=diffusion_configurator,
    checkpoint_path = checkpoint_path_uniform
)
amortizer_uniform.summary()

#history = trainer_c_uniform.train_online(epochs=100, iterations_per_epoch=1000, batch_size=32)


INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 5)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 278, 2)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:Shape of simulation non-batchable context: ()
INFO:root:Could not determine shape of simulation batchable context. Type appears to be non-array: <class 'list'>,                                    so make sure your input configurator takes cares of that!
INFO:root:Loaded loss history from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\uniform\history_100.pkl.
INFO:root:Networks loaded from C:\Users\u0145642\OneDrive - KU Leuven\Desktop\Robust amortized Bayesian inference\3 ddm\networks\uniform\ckpt-100
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.

Model: "amortized_posterior_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ddm_inference (InvertibleN  multiple                  434037    
 etwork)                                                         
                                                                 
 set_transformer_5 (SetTran  multiple                  46544     
 sformer)                                                        
                                                                 
Total params: 480581 (1.83 MB)
Trainable params: 480521 (1.83 MB)
Non-trainable params: 60 (240.00 Byte)
_________________________________________________________________
